## Import Json and Requests package

In [31]:
import json
import requests

## Define UserData

In [32]:
user_data = """{
    "tenancyName": "default",
    "usernameOrEmailAddress": "admin",
    "password": "December#2018"
    }"""

In [33]:
orchestrator_url = f"http://172.95.1.77"

## Get bearer token based on user data

In [34]:
def getUserToken(user_data):
    url = orchestrator_url + f"/api/Account/Authenticate"
    data = requests.post(url,json=json.loads(user_data))
    authentication_data = json.loads(data.text)
    token = "Bearer " + str(authentication_data["result"])
    return token

In [35]:
token = getUserToken(user_data)
token

'Bearer G8PvkpHpuwFFLEtX9Es3OG9XZGatqV8JJPMBbR-DYk3kl2nf3aCsgsaNDGB_nV804yGbFa-5EwJsJOe7gEoGHkHt2-awwcPKXu_43L93DsJkBGWYLZoia-LucGY6rFFXYdXiIwS8ixuwsPyqzzi8z7QvlqfC8rhc4Yp6uR9cUs2ErKhOG67dDwpVLvCI5Z4plkPyH2ZaXUabOY-ZZ9ttj2zMjR9S7loXlfVLJhXAzGGQ-CLeBkQCW0eg39G-ldsW5Y15ygsan3QtwL5RxMfkwMiXE4wrWewJxVn0IaMEyGIHc_iKb-_BP2IHlv4Z1d6n7AWc5i1TM8yx_qVSLe_udqQISEm40-UoaSw4SLz95YDoimPnXWOg62NiGFEjDZxPp10XQW4AgcxD2NbhpZeLn6PNVajnD1envE1N0JBEM-B8jLwkEYN1o0EUe9bAd3Nwo8g6oRGKR476jYvYNh0n__iNSs_4DnHt7uPCrYsJlXGCQWDNsxkVbHSft_XqZbi3egAl0OtfUs40imAtZQ5BVgrH0j0'

## Get ProcessID from ProcessName and bearer token

In [36]:
def getProcessID(process_name,token):
    url = orchestrator_url + f"/odata/Releases?$filter=ProcessKey+eq%20'{process_name}'"
    url
    process_data = requests.get(orchestrator_url + f"/odata/Releases?$filter=ProcessKey+eq%20'{process_name}'",
                                headers={"Authorization": token})
    process_json = json.loads(process_data.text)
    process_ID = process_json["value"][0]["Key"]
    return process_ID

In [37]:
process_ID = getProcessID("TestPublish",token)
process_ID

'f74d6ac6-525a-495f-bd4b-1e0ef0b78d6c'

## Get RobotID from RobotName and bearer token

In [38]:
def getRobotID(robotName, token):
    robot_name = requests.get(orchestrator_url + f"/odata/Robots?$filter=Name%20eq%20'{robotName}'",
                              headers={"Authorization": token})
    url = orchestrator_url + f"/odata/Robots?$filter=Name%20eq%20'{robotName}'"
    print(url)
    robot_json_obj = json.loads(robot_name.text)
    robot_ID = robot_json_obj["value"][0]["Id"]
    return robot_ID

In [39]:
robot_ID = getRobotID("MadRobo", token)
robot_ID

http://172.95.1.77/odata/Robots?$filter=Name%20eq%20'MadRobo'


2

## Run Job using ProcessID, RobotID and BearerToken

In [146]:
def runJob(process_ID, robot_ID, token):
    start_job_json = """{ "startInfo":
       { "ReleaseKey": \"""" + process_ID + """\",
         "Strategy": "Specific",
         "RobotIds": [ """ + str(robot_ID) + """ ],
         "Source": "Manual",
         "InputArguments": "{'in_Arg1':'Aloha'}"        
       } 
    }"""  # InputArguments should be left {} or not included if workflow does not accept any input
    start_job_data = requests.post(orchestrator_url + f"/odata/Jobs/UiPath.Server.Configuration.OData.StartJobs",
                                   json=json.loads(start_job_json), headers={"Authorization": token})
    return start_job_data

In [147]:
response_code = runJob(process_ID, robot_ID, token)
response_code

<Response [201]>

### Response code 201 donotes Job run is successful